In [0]:
%tensorflow_version 1.x
import sys
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelBinarizer
sys.path.append('/content/drive/My Drive/Colab_Work/PPM/nn/conv')
from minigooglenet import MiniGoogLeNet
sys.path.append('/content/drive/My Drive/Colab_Work/PPM/callbacks')
from trainingmonitor import TrainingMonitor
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

NUM_EPOCHS = 70
INIT_LR = 5e-3

In [6]:
def poly_decay(epoch):
	maxEpochs = NUM_EPOCHS
	baseLR = INIT_LR
	power=1.0

	alpha = baseLR * (1 - (epoch/float(maxEpochs))) ** power
	return float(alpha)

print("[INFO] loading CIFAR-10 data...")

((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float")
testX = testX.astype("float")

mean = np.mean(trainX, axis=0)
trainX -= mean
testX -= mean

trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

aug = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1,  
	    horizontal_flip=True, fill_mode="nearest")

[INFO] loading CIFAR-10 data...


In [7]:
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, momentum=0.9)
model = MiniGoogLeNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


In [9]:
callbacks = [LearningRateScheduler(poly_decay)]
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=64),
	    validation_data=(testX, testY), steps_per_epoch=len(trainX)//64, epochs=NUM_EPOCHS, 
	    callbacks=callbacks, verbose=1)

[INFO] training network...
Epoch 1/70
781/781 [==============================] - 33s 42ms/step - loss: 0.2240 - acc: 0.9235 - val_loss: 0.7185 - val_acc: 0.8267
Epoch 2/70
781/781 [==============================] - 33s 42ms/step - loss: 0.2075 - acc: 0.9283 - val_loss: 0.5896 - val_acc: 0.8373
Epoch 3/70
781/781 [==============================] - 33s 42ms/step - loss: 0.1889 - acc: 0.9347 - val_loss: 0.5342 - val_acc: 0.8529
Epoch 4/70
781/781 [==============================] - 33s 42ms/step - loss: 0.1761 - acc: 0.9383 - val_loss: 0.6491 - val_acc: 0.8312
Epoch 5/70
781/781 [==============================] - 33s 42ms/step - loss: 0.1646 - acc: 0.9432 - val_loss: 0.4853 - val_acc: 0.8703
Epoch 6/70
781/781 [==============================] - 33s 42ms/step - loss: 0.1540 - acc: 0.9457 - val_loss: 0.4886 - val_acc: 0.8588
Epoch 7/70
781/781 [==============================] - 33s 42ms/step - loss: 0.1488 - acc: 0.9473 - val_loss: 0.5794 - val_acc: 0.8479
Epoch 8/70
781/781 [===============

In [10]:
from sklearn.metrics import classification_report
labelNames = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

[INFO] evaluating network...
              precision    recall  f1-score   support

    airplane       0.91      0.93      0.92      1000
  automobile       0.96      0.96      0.96      1000
        bird       0.88      0.87      0.88      1000
         cat       0.84      0.80      0.82      1000
        deer       0.89      0.91      0.90      1000
         dog       0.87      0.85      0.86      1000
        frog       0.90      0.94      0.92      1000
       horse       0.94      0.92      0.93      1000
        ship       0.95      0.94      0.95      1000
       truck       0.94      0.95      0.95      1000

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000



In [11]:
print("[INFO] serializing network...")
%cd /content/drive/My\ Drive/Colab_Work
model.save("minigooglenet_cifar10.hdf5")
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 70), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 70), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 70), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 70), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
# %cd /content/drive/My\ Drive/Colab_Work
plt.savefig('minigooglenet_cifar10_train_test_plot.png')

[INFO] serializing network...
/content/drive/My Drive/Colab_Work
